In [1]:
#https://www.kaggle.com/c/cifar-10


pip install kaggle-cli
#get data from kaggle
kg download -u <username> -p <password> -c cifar-10

p7zip -d train.7z  
p7zip -d test.7z  


In [2]:
from datetime import datetime
HOMEPATH = "/home/ubuntu/fastai/"
DATA_HOME_DIR = HOMEPATH + "data/cifar-10/"
sampleRatio = 0.2
trainRatio = 0.8
import os

In [3]:
from keras import backend as K
import os

print("----------------")
def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


----------------


In [4]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
import utils; reload(utils)
from utils import *

('os.getcwd:', '/home/ubuntu/fastai')


In [5]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import pandas as pd
import scipy
from random import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

('os.getcwd:', '/home/ubuntu/fastai')


In [6]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [7]:
#path = HOMEPATH + "data/dogscats/sample/"
path = DATA_HOME_DIR 
#path = DATA_HOME_DIR + "sample/"
train_path = path + 'train/'
test_path = path + 'test/'
testUnknown_path = path + 'test/unknown/'
valid_path = path + 'valid/'
model_path = path + 'models/'
sample_path = path + 'sample/'
sampleTrain_path = path + 'sample/train/'
sampleTest_path = path + 'sample/test/'
sampleValid_path = path + 'sample/valid/'
sampleResults_path = path + 'sample/results/'
sampleTestUnknown_path = path + 'sample/test/unknown/'

categories = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def makeNewDir(newDirPath):
    if not os.path.exists(newDirPath): 
        print("directory ", newDirPath, " did not exist, creating:")
        os.mkdir(newDirPath)
    else:
        print("directory ", newDirPath, "already existed.")

#makeNewDir(train_path)#already exists after unzipping
#makeNewDir(test_path)#already exists after unzipping

makeNewDir(testUnknown_path)
makeNewDir(valid_path)
makeNewDir(model_path)
makeNewDir(sample_path)
makeNewDir(sampleTrain_path)
makeNewDir(sampleTest_path)
makeNewDir(sampleValid_path)
makeNewDir(sampleResults_path)
makeNewDir(sampleTestUnknown_path)
for category in categories:
    newDir = train_path + category
    makeNewDir(newDir)
    newDir = valid_path + category
    makeNewDir(newDir)
    newDir = sampleTrain_path + category
    makeNewDir(newDir)
    newDir = sampleValid_path + category
    makeNewDir(newDir)



directory  /home/ubuntu/fastai/data/cifar-10/test/unknown/ already existed.
directory  /home/ubuntu/fastai/data/cifar-10/valid/  did not exist, creating:
directory  /home/ubuntu/fastai/data/cifar-10/models/  did not exist, creating:
directory  /home/ubuntu/fastai/data/cifar-10/sample/  did not exist, creating:
directory  /home/ubuntu/fastai/data/cifar-10/sample/train/  did not exist, creating:
directory  /home/ubuntu/fastai/data/cifar-10/sample/test/  did not exist, creating:
directory  /home/ubuntu/fastai/data/cifar-10/sample/valid/  did not exist, creating:
directory  /home/ubuntu/fastai/data/cifar-10/sample/results/  did not exist, creating:
directory  /home/ubuntu/fastai/data/cifar-10/sample/test/unknown/  did not exist, creating:
directory  /home/ubuntu/fastai/data/cifar-10/train/airplane  did not exist, creating:
directory  /home/ubuntu/fastai/data/cifar-10/valid/airplane  did not exist, creating:
directory  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane  did not exist, 

In [8]:
#load trainLabels.csv and use to move files to category directories
trainLabelsFile = DATA_HOME_DIR + "trainLabels.csv"
trainLabels = pd.read_csv(trainLabelsFile).as_matrix()
print (type(trainLabels), trainLabels.shape)
print (trainLabels[0:10,])

<type 'numpy.ndarray'> (50000, 2)
[[1 'frog']
 [2 'truck']
 [3 'truck']
 [4 'deer']
 [5 'automobile']
 [6 'automobile']
 [7 'bird']
 [8 'horse']
 [9 'ship']
 [10 'cat']]


In [9]:
#
def moveFilesToCategories(path_):
    os.chdir(path_)
    errCount = 0
    success = 0
    print ("current dir:", os.getcwd())
    for i in range(0,trainLabels.shape[0]):
        #move file to subdirectory 
        #print (trainLabels[i,0], trainLabels[i,1])
        #print("moving:", str(trainLabels[i,0])+".png", " to ", trainLabels[i,1]+"/"+str(trainLabels[i,0])+".png")
        try:
            os.rename(str(trainLabels[i,0])+".png", trainLabels[i,1]+"/"+str(trainLabels[i,0])+".png")
            success += 1
        except:
            errCount += 1
            #print ("Unexpected error:", sys.exc_info()[0])
        #if i>5: break
    print ("success:", success)
    print ("errCount:", errCount)

In [10]:
moveFilesToCategories(train_path)

current dir: /home/ubuntu/fastai/data/cifar-10/train
success: 50000
errCount: 0


In [11]:
def dirFileList(dir_path):
    return [name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, name))]
    

In [12]:
def moveListToDir(fileList, currDir, destDir):
    print ("moving ", len(fileList), " files from ", currDir, " to ", destDir)
    for file in fileList:
        #move file to destDir
        errCount = 0
        success = 0
        try:
            os.rename(currDir+"/"+file, destDir+"/"+file)
            print (currDir+"/"+file, " to ", destDir+"/"+file)
            success += 1
        except:
            errCount += 1
            print ("error moving ", currDir+"/"+file, " to ", destDir+"/"+file)
            print ("Unexpected error:", sys.exc_info()[0])
        #break
    print ("success:", success)
    print ("errCount:", errCount)


In [13]:
print (train_path, " has ", len(dirFileList(train_path)), " files.")
#identify files in each category.
#apply split for train | sample, sample/train, sample/valid and valid
for category in categories:
    fileList = dirFileList(train_path+category)
    print (train_path+category, " has ", len(fileList), " files.")
    #randomly select fraction (sampleRatio) of files to move to sample
    shuffle(fileList)
    numSamples = int(len(fileList)*sampleRatio)
    sampleList = fileList[0:numSamples]
    print ("sampleList:", len(sampleList))
    nonValidList =  fileList[numSamples:]
    print ("nonValidList:", len(nonValidList))

    numTrain = int(len(nonValidList)*trainRatio)
    trainList = nonValidList[0:numTrain]#these don't move 
    validList = nonValidList[numTrain:]#previous version had split mixed up.
    print ("trainList:", len(trainList))
    print ("validList:", len(validList))
    moveListToDir(validList, train_path+category, valid_path+category)
    
    #  of this selection, move randomly selected fraction (trainValidRatio)   to sample/valid 
    numSamplesTrain = int(len(sampleList)*trainRatio)
    sampleTrainList = sampleList[0:numSamplesTrain]
    sampleValidList = sampleList[numSamplesTrain:]
    print ("sampleTrainList:", len(sampleTrainList))
    moveListToDir(sampleTrainList, train_path+category, sampleTrain_path+category)
    print ("sampleValidList:", len(sampleValidList))
    moveListToDir(sampleValidList, train_path+category, sampleValid_path+category)


/home/ubuntu/fastai/data/cifar-10/train/  has  0  files.
/home/ubuntu/fastai/data/cifar-10/train/airplane  has  5000  files.
sampleList: 1000
nonValidList: 4000
trainList: 3200
validList: 800
moving  800  files from  /home/ubuntu/fastai/data/cifar-10/train/airplane  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane
/home/ubuntu/fastai/data/cifar-10/train/airplane/13111.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/13111.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/5624.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/5624.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/24170.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/24170.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/42845.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/42845.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/37673.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/37673.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/47221.png  to  /ho

/home/ubuntu/fastai/data/cifar-10/train/airplane/49892.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/49892.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/37356.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/37356.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/35326.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/35326.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/29161.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/29161.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/31594.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/31594.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/117.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/117.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/41976.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/41976.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/40704.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/40704.png
/home/ubuntu/fastai/

/home/ubuntu/fastai/data/cifar-10/train/airplane/26723.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/26723.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/24000.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/24000.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/14170.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/14170.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/25996.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/25996.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/15708.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/15708.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/29941.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/29941.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/29918.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/29918.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/16580.png  to  /home/ubuntu/fastai/data/cifar-10/valid/airplane/16580.png
/home/ubuntu/fas

/home/ubuntu/fastai/data/cifar-10/train/airplane/4080.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/4080.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/44246.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/44246.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/6744.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/6744.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/10487.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/10487.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/37951.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/37951.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/49993.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/49993.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/11277.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/11277.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/47321.png  to  /home/ubuntu/fastai/data/cifar

/home/ubuntu/fastai/data/cifar-10/train/airplane/374.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/374.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/34712.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/34712.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/31005.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/31005.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/36338.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/36338.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/44028.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/44028.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/37104.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/37104.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/5471.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/airplane/5471.png
/home/ubuntu/fastai/data/cifar-10/train/airplane/5588.png  to  /home/ubuntu/fastai/data/cifar-10

...

/home/ubuntu/fastai/data/cifar-10/train/ship/15689.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/15689.png
/home/ubuntu/fastai/data/cifar-10/train/ship/35110.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/35110.png
/home/ubuntu/fastai/data/cifar-10/train/ship/44318.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/44318.png
/home/ubuntu/fastai/data/cifar-10/train/ship/3171.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/3171.png
/home/ubuntu/fastai/data/cifar-10/train/ship/12651.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/12651.png
/home/ubuntu/fastai/data/cifar-10/train/ship/11694.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/11694.png
/home/ubuntu/fastai/data/cifar-10/train/ship/45025.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/45025.png
/home/ubuntu/fastai/data/cifar-10/train/ship/19465.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/19465.png
/home/ubuntu/fastai/data/c

/home/ubuntu/fastai/data/cifar-10/train/ship/16207.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/16207.png
/home/ubuntu/fastai/data/cifar-10/train/ship/47115.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/47115.png
/home/ubuntu/fastai/data/cifar-10/train/ship/23961.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/23961.png
/home/ubuntu/fastai/data/cifar-10/train/ship/3635.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/3635.png
/home/ubuntu/fastai/data/cifar-10/train/ship/47507.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/47507.png
/home/ubuntu/fastai/data/cifar-10/train/ship/18951.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/18951.png
/home/ubuntu/fastai/data/cifar-10/train/ship/24812.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/24812.png
/home/ubuntu/fastai/data/cifar-10/train/ship/18396.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/ship/18396.png
/home/ubuntu/fastai/data/c

/home/ubuntu/fastai/data/cifar-10/train/truck/21508.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/21508.png
/home/ubuntu/fastai/data/cifar-10/train/truck/48285.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/48285.png
/home/ubuntu/fastai/data/cifar-10/train/truck/11802.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/11802.png
/home/ubuntu/fastai/data/cifar-10/train/truck/11787.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/11787.png
/home/ubuntu/fastai/data/cifar-10/train/truck/34864.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/34864.png
/home/ubuntu/fastai/data/cifar-10/train/truck/42386.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/42386.png
/home/ubuntu/fastai/data/cifar-10/train/truck/17.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/17.png
/home/ubuntu/fastai/data/cifar-10/train/truck/22443.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/22443.png
/home/ubuntu/fastai/data/cifar-10/train/truck/46105.png  to  /home/ubu

/home/ubuntu/fastai/data/cifar-10/train/truck/22644.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/22644.png
/home/ubuntu/fastai/data/cifar-10/train/truck/3922.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/3922.png
/home/ubuntu/fastai/data/cifar-10/train/truck/32147.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/32147.png
/home/ubuntu/fastai/data/cifar-10/train/truck/1749.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/1749.png
/home/ubuntu/fastai/data/cifar-10/train/truck/11140.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/11140.png
/home/ubuntu/fastai/data/cifar-10/train/truck/38072.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/38072.png
/home/ubuntu/fastai/data/cifar-10/train/truck/9047.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/9047.png
/home/ubuntu/fastai/data/cifar-10/train/truck/21610.png  to  /home/ubuntu/fastai/data/cifar-10/valid/truck/21610.png
/home/ubuntu/fastai/data/cifar-10/train/truck/29436.png  to  /home/ubu

/home/ubuntu/fastai/data/cifar-10/train/truck/13422.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/truck/13422.png
/home/ubuntu/fastai/data/cifar-10/train/truck/45021.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/truck/45021.png
/home/ubuntu/fastai/data/cifar-10/train/truck/20519.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/truck/20519.png
/home/ubuntu/fastai/data/cifar-10/train/truck/750.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/truck/750.png
/home/ubuntu/fastai/data/cifar-10/train/truck/5603.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/truck/5603.png
/home/ubuntu/fastai/data/cifar-10/train/truck/43347.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/truck/43347.png
/home/ubuntu/fastai/data/cifar-10/train/truck/26586.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/truck/26586.png
/home/ubuntu/fastai/data/cifar-10/train/truck/10794.png  to  /home/ubuntu/fastai/data/cifar-10/sample/train/truck/10794.png
/home/ubuntu/f

In [14]:
dirList = [train_path, valid_path, sampleTrain_path, sampleValid_path]

for dir_path in dirList:
    for category in categories:
        print ("# of files in", dir_path+category, ":", len(dirFileList(dir_path+category)))


# of files in /home/ubuntu/fastai/data/cifar-10/train/airplane : 3200
# of files in /home/ubuntu/fastai/data/cifar-10/train/automobile : 3200
# of files in /home/ubuntu/fastai/data/cifar-10/train/bird : 3200
# of files in /home/ubuntu/fastai/data/cifar-10/train/cat : 3200
# of files in /home/ubuntu/fastai/data/cifar-10/train/deer : 3200
# of files in /home/ubuntu/fastai/data/cifar-10/train/dog : 3200
# of files in /home/ubuntu/fastai/data/cifar-10/train/frog : 3200
# of files in /home/ubuntu/fastai/data/cifar-10/train/horse : 3200
# of files in /home/ubuntu/fastai/data/cifar-10/train/ship : 3200
# of files in /home/ubuntu/fastai/data/cifar-10/train/truck : 3200
# of files in /home/ubuntu/fastai/data/cifar-10/valid/airplane : 800
# of files in /home/ubuntu/fastai/data/cifar-10/valid/automobile : 800
# of files in /home/ubuntu/fastai/data/cifar-10/valid/bird : 800
# of files in /home/ubuntu/fastai/data/cifar-10/valid/cat : 800
# of files in /home/ubuntu/fastai/data/cifar-10/valid/deer : 

In [15]:
#park here. next step will load files from pre-prepared directory structure into model 